<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [3]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [4]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [5]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [6]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [7]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [8]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [9]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [10]:
!rm -rf logs

In [11]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 22s 841ms/step - loss: 0.6749 - accuracy: 0.5523 - val_loss: 0.6653 - val_accuracy: 0.5220
Epoch 2/10
26/26 [==============================] - 20s 781ms/step - loss: 0.6312 - accuracy: 0.6618 - val_loss: 0.6055 - val_accuracy: 0.7805
Epoch 3/10
26/26 [==============================] - 20s 788ms/step - loss: 0.6068 - accuracy: 0.7226 - val_loss: 0.5581 - val_accuracy: 0.8439
Epoch 4/10
26/26 [==============================] - 20s 765ms/step - loss: 0.5476 - accuracy: 0.7555 - val_loss: 0.5855 - val_accuracy: 0.6390
Epoch 5/10
26/26 [==============================] - 20s 751ms/step - loss: 0.5184 - accuracy: 0.7749 - val_loss: 0.4514 - val_accuracy: 0.8634
Epoch 6/10
26/26 [==============================] - 20s 757ms/step - loss: 0.4355 - accuracy: 0.8333 - val_loss: 0.3971 - val_accuracy: 0.8683
Epoch 7/10
26/26 [==============================] - 20s 754ms/step - loss: 0.4250 - accuracy: 0.8418 - val_loss: 0.3202 - val_accuracy: 0.9512

In [12]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.61.h5
26/26 - 21s - loss: 0.6534 - accuracy: 0.6119 - val_loss: 0.6077 - val_accuracy: 0.7561
Epoch 2/5

Epoch 00002: saving model to weights.02-0.56.h5
26/26 - 20s - loss: 0.5785 - accuracy: 0.7165 - val_loss: 0.5554 - val_accuracy: 0.7805
Epoch 3/5

Epoch 00003: saving model to weights.03-0.48.h5
26/26 - 20s - loss: 0.5051 - accuracy: 0.7664 - val_loss: 0.4761 - val_accuracy: 0.7561
Epoch 4/5

Epoch 00004: saving model to weights.04-0.42.h5
26/26 - 20s - loss: 0.4688 - accuracy: 0.7689 - val_loss: 0.4243 - val_accuracy: 0.7902
Epoch 5/5

Epoch 00005: saving model to weights.05-0.41.h5
26/26 - 20s - loss: 0.3950 - accuracy: 0.8248 - val_loss: 0.4137 - val_accuracy: 0.8537


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6750 - accuracy: 0.5547 - val_loss: 0.6350 - val_accuracy: 0.7805


In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 21s - loss: 0.6605 - accuracy: 0.6241 - val_loss: 0.6255 - val_accuracy: 0.6780
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.5974 - accuracy: 0.7032 - val_loss: 0.5860 - val_accuracy: 0.7220


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6757 - accuracy: 0.5900 - val_loss: 0.6509 - val_accuracy: 0.7854
Epoch 2/50
26/26 - 20s - loss: 0.6385 - accuracy: 0.6314 - val_loss: 0.5982 - val_accuracy: 0.7805
Epoch 3/50
26/26 - 20s - loss: 0.5892 - accuracy: 0.6959 - val_loss: 0.5429 - val_accuracy: 0.8049
Epoch 4/50
26/26 - 20s - loss: 0.5324 - accuracy: 0.7530 - val_loss: 0.5429 - val_accuracy: 0.7268
Epoch 5/50
26/26 - 19s - loss: 0.4667 - accuracy: 0.7956 - val_loss: 0.8446 - val_accuracy: 0.5024
Epoch 6/50
26/26 - 20s - loss: 0.4346 - accuracy: 0.8066 - val_loss: 0.4162 - val_accuracy: 0.8439
Epoch 7/50
26/26 - 18s - loss: 0.3616 - accuracy: 0.8577 - val_loss: 0.2973 - val_accuracy: 0.8878
Epoch 8/50
26/26 - 19s - loss: 0.3393 - accuracy: 0.8589 - val_loss: 0.2777 - val_accuracy: 0.9122
Epoch 9/50
26/26 - 19s - loss: 0.2786 - accuracy: 0.9088 - val_loss: 0.2946 - val_accuracy: 0.8634
Epoch 10/50
26/26 - 19s - loss: 0.2322 - accuracy: 0.9258 - val_loss: 0.1894 - val_accuracy: 0.9463
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [17]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 824ms/step - loss: 0.6702 - accuracy: 0.5827 - val_loss: 0.6520 - val_accuracy: 0.5659
Epoch 2/5
26/26 [==============================] - 20s 779ms/step - loss: 0.6286 - accuracy: 0.6411 - val_loss: 0.5851 - val_accuracy: 0.8244
Epoch 3/5
26/26 [==============================] - 20s 769ms/step - loss: 0.5802 - accuracy: 0.7141 - val_loss: 0.5329 - val_accuracy: 0.8049
Epoch 4/5
26/26 [==============================] - 20s 774ms/step - loss: 0.5260 - accuracy: 0.7433 - val_loss: 0.7428 - val_accuracy: 0.5854
Epoch 5/5
26/26 [==============================] - 20s 776ms/step - loss: 0.5075 - accuracy: 0.7579 - val_loss: 0.5740 - val_accuracy: 0.6683


In [18]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.582725,0.670556,0.565854,0.652035
1,1,0.641119,0.628647,0.824390,0.585073
2,2,0.714112,0.580776,0.804878,0.532864
3,3,0.743309,0.524040,0.585366,0.742800
4,4,0.757908,0.506412,0.668293,0.573966


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [19]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 29s 1s/step - loss: 0.6793 - accuracy: 0.5560 - val_loss: 0.6802 - val_accuracy: 0.4585

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 21s 789ms/step - loss: 0.6529 - accuracy: 0.6448 - val_loss: 0.6667 - val_accuracy: 0.5171

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 774ms/step - loss: 0.6425 - accuracy: 0.6557 - val_loss: 0.6602 - val_accuracy: 0.5366

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 785ms/step - loss: 0.6346 - accuracy: 0.6618 - val_loss: 0.6501 - val_accuracy: 0.5951

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 773ms/step - loss: 0.6321

In [24]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 5285), started 0:25:25 ago. (Use '!kill 5285' to kill it.)

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [21]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 827ms/step - loss: 0.6767 - accuracy: 0.5779 - val_loss: 0.6750 - val_accuracy: 0.5171
Epoch 2/50
25/26 [===========================>..] - ETA: 0s - loss: 0.6235 - accuracy: 0.6875
Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 20s 763ms/step - loss: 0.6242 - accuracy: 0.6825 - val_loss: 0.6778 - val_accuracy: 0.5122
Epoch 3/50
26/26 [==============================] - 20s 758ms/step - loss: 0.5859 - accuracy: 0.7433 - val_loss: 0.5917 - val_accuracy: 0.7707
Epoch 4/50
26/26 [==============================] - 19s 715ms/step - loss: 0.5646 - accuracy: 0.7749 - val_loss: 0.5654 - val_accuracy: 0.7902
Epoch 5/50
26/26 [==============================] - 19s 741ms/step - loss: 0.5365 - accuracy: 0.7895 - val_loss: 0.5532 - val_accuracy: 0.7854
Epoch 7/50
25/26 [===========================>..] - ETA: 0s - loss: 0.5277 - accuracy: 0.8025
Epoch 00007: ReduceLROnPlate

In [22]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 5285), started 0:25:00 ago. (Use '!kill 5285' to kill it.)